#### Import Abstracts

In [11]:
import pandas as pd
data = pd.read_csv("Abstracts-186.csv")
data = data.drop('Unnamed: 0', axis=1)
data

,Entity,Label,Abstract
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A..."
...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...


#### Model

In [ ]:
pip install git+https://github.com/mmxgn/spacy-clausie.git

In [12]:
import claucy 
import spacy

chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
#nlp_extract = spacy.load("en_core_web_sm")
nlp_sub = spacy.load("en_core_web_sm")
claucy.add_to_pipe(nlp) 
datatrip = []
j = 0
namest = 0
    
for s,label,abstract in data.values:

    if type(abstract) is float:
        datatrip.append([])
        print(j,label)
        j += 1
        continue

    print(j,label)
    doc = nlp(abstract)

    for sent in doc.sents:

        if len(sent._.clauses) == 0:
            continue
      
        propositions = sent._.clauses[0].to_propositions(as_text=False, inflect=None)  
        for triple in propositions:
            if namest == 0:
                name = str(triple[0])
                namest += 1
            if len(triple) == 3:
                sub = nlp_sub(str(triple[0]))
                tokenlist = []
                for token in sub:
                    if token.pos_ == 'NOUN':
                        tokenlist.append(token)
                if any(str(triple[0]).casefold() == str(item) for item in item_list) or any(str(item).casefold() in str(label).casefold() for item in tokenlist) or str(triple[0]) in name or str(triple[0]) in label:
                    trips.append([str(triple[1]), str(triple[2])])
    
    datatrip.append(trips)
    namest = 0
    trips = []
    j += 1

0 David Bowie
1 Kanye West
2 Eminem
3 Jennifer Aniston
4 George W. Bush
5 Prince (musician)
6 Justin Bieber
7 Drake (musician)
8 Heath Ledger
9 John Krasinski
10 Val Kilmer
11 American Horror Story
12 Brad Pitt
13 Japan
14 Barack Obama
15 Bill Clinton
16 Donald Trump
17 Jimmy Carter
18 Martin Luther King Jr.
19 Ronald Reagan
20 Germany
21 Matthew Perry
22 Vietnam War
23 The Walking Dead (TV series)
24 London
25 World War I
26 California
27 The Beatles
28 Robert Downey Jr.
29 Roger Federer
30 Marilyn Manson
31 Ben Affleck
32 India
33 Serena Williams
34 Novak Djokovic
35 China
36 France
37 Michael Jackson
38 Nicolas Cage
39 Johnny Depp
40 South Africa
41 Tobey Maguire
42 Sandra Bullock
43 Paul Walker
44 Mark Wahlberg
45 New Zealand
46 Alec Baldwin
47 Rachel Weisz
48 Stanley Tucci
49 Keanu Reeves
50 Frank Sinatra
51 Salma Hayek
52 Ryan Reynolds
53 Snoop Dogg
54 Priyanka Chopra
55 Ernest Hemingway
56 Brittany Murphy
57 Lady Gaga
58 Robin Williams
59 Will Smith
60 Mila Kunis
61 Quentin Tara

In [13]:
data['ClCy-Trips-Raw'] = datatrip
data
#if save needed
#data.to_csv('ClCyPt1.csv')

,Entity,Label,Abstract,ClCy-Trips-Raw
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...,"[[is, January], [was acclaimed, by critics and..."
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...,"[[is, /jeɪ/ YAY], [regarded, widely], [regarde..."
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1...","[[is, an American rapper and record producer],..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196...","[[is, film producer], [is, an American actress..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...","[[is, an American politician], [served, from 1..."
...,...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...,"[[is, née D'Alesandro], [has represented, Cali..."
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te...","[[is, a state], [is, populous state], [is, the..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...,"[[is, a state], [is, the second-most populous ..."
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...,"[[is, French pronunciation], [compete, the top..."


In [14]:
from transformers import pipeline
import spacy
import ast

unmasker = pipeline('fill-mask', model='bert-base-uncased', top_k=1, framework='pt', device = -1)
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
#import pandas as pd
#data = pd.read_csv("ClCyPt1.csv")
#data = data.drop('Unnamed: 0', axis=1)
#data

In [15]:
import ast
from transformers import pipeline
import spacy

unmasker = pipeline('fill-mask', model='bert-base-uncased', top_k=1, framework='pt',device=-1)
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0
trips = []
history = []
for s,label,abstract,triples in data.values[j:]:
    
    if type(abstract) is float:
            datatrips.append([])
            print(j,label, 'nothing')
            j += 1
            continue
            
    print(j,label)
    for triple in triples:
    #for triple in ast.literal_eval(triples):
        # turn triple into string 
        string = (label,triple[0],triple[1])
        string = (' '.join(string))
        tokens = nlp(string)

        for chunk in tokens.noun_chunks:
            chunks.append(chunk.root.head.text)

        # if something in chunk
        if len(chunks) > 0:

            if chunks[-1] == 'is' or chunks[-1] == 'was':
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    if ([chunks[-1],b.text]) not in history:
                      #print('ye - 1',[chunks[-1],b.text])
                        history.append([chunks[-1],b.text])
                        slm = ("%s. %s %s [MASK] %s" %(string, label, chunks[-1], b.text))
                    # check not already in triples
                    #if ([unmasker(slm)[0]['token_str'], b.text]) not in trips:
                        trips.append([unmasker(slm)[0]['token_str'], b.text])

            else:
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    slm = ("%s. %s [MASK] %s %s" %(string, label, chunks[-1], b.text))
                    if ([chunks[-1],b.text]) not in history:
                      #print('ye-',[chunks[-1],b.text])
                        history.append([chunks[-1],b.text])
                        mask = unmasker(slm)[0]['token_str']
                    # if MASK is punctuation, remove MASK
                        if mask in punc:
                          #if ([chunks[-1], b.text]) not in trips:
                              # check not already in triples
                            trips.append([chunks[-1], b.text])

                        else:
                          #if (["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text]) not in trips:
                              # check not already in triples
                            trips.append(["%s%s" %(mask, chunks[-1].capitalize()),b.text])

        else:
            trips.append(['%s' %(triple[0]),'%s' %(triple[1])])
            # empty chunk list
        chunks = []
    #print(trips)
    datatrips.append(trips)
    trips = []
    history = []
    j += 1

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 David Bowie
1 Kanye West
2 Eminem
3 Jennifer Aniston
4 George W. Bush
5 Prince (musician)
6 Justin Bieber
7 Drake (musician)
8 Heath Ledger
9 John Krasinski
10 Val Kilmer
11 American Horror Story
12 Brad Pitt
13 Japan
14 Barack Obama
15 Bill Clinton
16 Donald Trump
17 Jimmy Carter
18 Martin Luther King Jr.
19 Ronald Reagan
20 Germany
21 Matthew Perry
22 Vietnam War
23 The Walking Dead (TV series)
24 London
25 World War I
26 California
27 The Beatles
28 Robert Downey Jr.
29 Roger Federer
30 Marilyn Manson
31 Ben Affleck
32 India
33 Serena Williams
34 Novak Djokovic
35 China
36 France
37 Michael Jackson
38 Nicolas Cage
39 Johnny Depp
40 South Africa
41 Tobey Maguire
42 Sandra Bullock
43 Paul Walker
44 Mark Wahlberg
45 New Zealand
46 Alec Baldwin
47 Rachel Weisz
48 Stanley Tucci
49 Keanu Reeves
50 Frank Sinatra
51 Salma Hayek
52 Ryan Reynolds
53 Snoop Dogg
54 Priyanka Chopra
55 Ernest Hemingway
56 Brittany Murphy
57 Lady Gaga
58 Robin Williams
59 Will Smith
60 Mila Kunis
61 Quentin Tara

In [8]:
# 12:57 - 17:00. ::::: 4 hours
# 10 mins

In [16]:
data['ClCy'] = datatrips
data
# save if needed 
#data.to_csv('ClCyPt2')

,Entity,Label,Abstract,ClCy-Trips-Raw,ClCy
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...,"[[is, January], [was acclaimed, by critics and...","[[in, January], [musicCritics, critics], [crit..."
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...,"[[is, /jeɪ/ YAY], [regarded, widely], [regarde...","[[also, /jeɪ/ YAY], [namedArtists, the most in..."
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1...","[[is, an American rapper and record producer],...","[[also, an American rapper and record producer..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196...","[[is, film producer], [is, an American actress...","[[a, film producer], [also, an American actres..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...","[[is, an American politician], [served, from 1...","[[also, an American politician], [sonOf, the 4..."
...,...,...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...,"[[is, née D'Alesandro], [has represented, Cali...","[[also, née D'Alesandro], [hasRepresented, Cal..."
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te...","[[is, a state], [is, populous state], [is, the...","[[not, a state], [most, populous state], [also..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...,"[[is, a state], [is, the second-most populous ...","[[not, a state], [also, the second-most populo..."
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...,"[[is, French pronunciation], [compete, the top...","[[the, French pronunciation], [alsoCompete, th..."


# final processing 

In [21]:
final_trips = []
import nltk
import ast
trips = []
k = 0
for s,label,abstract,_,triples in data.values:
    print(k)
    k+=1
    if type(abstract) == float:
        print('ye')
        final_trips.append([])
        continue
    
    #for triple in ast.literal_eval(triples):
    for triple in triples:
        if triple[0] == 'is' or triple[0] == 'also' or triple[0] == 'an' or triple[0] == 'a' or triple[0] == 'was' or triple[0] == 'as' or triple[0] == 'not' or triple[0] == 'the':
            simpletok = str(triple[1]).split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(triple[0],'- type'," ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append(['type'," ".join(simpletok[1:])])
            else:
                print(triple[0], '-', 'type', triple[1], '-', label)
                trips.append(['type', triple[1]])
        else:
            simpletok = str(triple[1]).split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(" ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append([triple[0], " ".join(simpletok[1:])])
            else:
                trips.append(triple)
    final_trips.append(trips)
    trips = []

0
interest - David Bowie
period - David Bowie
hard rock band - David Bowie
decade-long recording hiatus - David Bowie
release - David Bowie
Rock and Roll Hall - David Bowie
also - type best-selling vinyl artist - David Bowie
1
also - type /jeɪ/ YAY - Kanye West
most influential hip hop artists - Kanye West
record label - Kanye West
joint record - Kanye West
 - Kanye West
Kanye West clothing - Removed
fashion designer - Kanye West
also - type founder - Kanye West
the - type head - Kanye West
unsuccessful independent presidential campaign - Kanye West
series - Kanye West
2
also - type American rapper and record producer - Eminem
release - Eminem
extended play - Eminem
release - Eminem
music industry - Eminem
release - Eminem
US number one albums - Eminem
not - type member - Eminem
best-selling music artists - Eminem
best-selling male music artist - Eminem
United States - Eminem
also - type best-selling music artist - Eminem
Rock and Roll Hall - Eminem
3
a - type film producer - Jennifer 

In [22]:
len(final_trips)

184

In [23]:
import re
trips = []
fin = []
for triples in final_trips:
    for triple in triples:
        pred = re.sub( r"([A-Z])", r" \1", triple[0]).split()
        if len(pred) > 1:
            if pred[0] == 'also' or pred[0] == 'was' or pred[0] == 'also' or pred[0] == 'has':
                relation = pred[1].lower()
                simpletok = str(triple[1]).split(' ')
                if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also' or simpletok[0] == 'his' or simpletok[0] == 'she':
                    if len(" ".join(simpletok[1:])) == 0:
                        continue
                        #print(triple[0], triple[1], '- Removed' )
                        #print()
                    else:
                        #print('fix')
                        #print(triple)
                        #print(relation," ".join(simpletok[1:]))
                        #print()
                        trips.append([relation," ".join(simpletok[1:])])
                else:
                    trips.append([relation,triple[1]])
            else:
                trips.append(triple)
        else:
            trips.append(triple)
    fin.append(trips)
    trips = []
fin

[[['in', 'January'],
  ['musicCritics', 'critics'],
  ['critics', 'musicians'],
  ['for', 'his innovative work'],
  ['developed', 'interest'],
  ['studied', 'art'],
  ['studied', 'music'],
  ['outOf', 'period'],
  ['periodOf', 'experimentation'],
  ['starred', 'The Man'],
  ['changed', 'direction'],
  ['achieved', 'massive commercial success'],
  ['fronted', 'hard rock band'],
  ['continuedContinued', 'experiment'],
  ['returnedWith', 'The Next Day'],
  ['returnedFrom', 'decade-long recording hiatus'],
  ['liveAlbum', 'both his 69th birthday'],
  ['soloAlbum', 'release'],
  ['album', 'his final album'],
  ['tributeAlbum', 'Blackstar'],
  ['awarded', 'album certifications'],
  ['awarded', 'platinum'],
  ['awarded', 'gold'],
  ['received', 'platinum'],
  ['received', 'gold certifications'],
  ['memberOf', 'Rock and Roll Hall'],
  ['hallOf', 'Fame'],
  ['type', 'best-selling vinyl artist']],
 [['type', '/jeɪ/ YAY'],
  ['namedArtists', 'most influential hip hop artists'],
  ['recordingArti

In [24]:
trips = []
final = []
for triples in fin:
    for triple in triples:
        obj = str(triple[1]).split(" ")
        obj = "_".join(obj)
        trips.append([triple[0], obj])
    final.append(trips)
    trips = []
len(final), final[:5]

(184,
 [[['in', 'January'],
   ['musicCritics', 'critics'],
   ['critics', 'musicians'],
   ['for', 'his_innovative_work'],
   ['developed', 'interest'],
   ['studied', 'art'],
   ['studied', 'music'],
   ['outOf', 'period'],
   ['periodOf', 'experimentation'],
   ['starred', 'The_Man'],
   ['changed', 'direction'],
   ['achieved', 'massive_commercial_success'],
   ['fronted', 'hard_rock_band'],
   ['continuedContinued', 'experiment'],
   ['returnedWith', 'The_Next_Day'],
   ['returnedFrom', 'decade-long_recording_hiatus'],
   ['liveAlbum', 'both_his_69th_birthday'],
   ['soloAlbum', 'release'],
   ['album', 'his_final_album'],
   ['tributeAlbum', 'Blackstar'],
   ['awarded', 'album_certifications'],
   ['awarded', 'platinum'],
   ['awarded', 'gold'],
   ['received', 'platinum'],
   ['received', 'gold_certifications'],
   ['memberOf', 'Rock_and_Roll_Hall'],
   ['hallOf', 'Fame'],
   ['type', 'best-selling_vinyl_artist']],
  [['type', '/jeɪ/_YAY'],
   ['namedArtists', 'most_influential_

In [25]:
data['ClCy'] = final
data

,Entity,Label,Abstract,ClCy-Trips-Raw,ClCy
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...,"[[is, January], [was acclaimed, by critics and...","[[in, January], [musicCritics, critics], [crit..."
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...,"[[is, /jeɪ/ YAY], [regarded, widely], [regarde...","[[type, /jeɪ/_YAY], [namedArtists, most_influe..."
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1...","[[is, an American rapper and record producer],...","[[type, American_rapper_and_record_producer], ..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196...","[[is, film producer], [is, an American actress...","[[type, film_producer], [type, American_actres..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...","[[is, an American politician], [served, from 1...","[[type, American_politician], [sonOf, 46th_gov..."
...,...,...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...,"[[is, née D'Alesandro], [has represented, Cali...","[[type, née_D'Alesandro], [represented, Califo..."
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te...","[[is, a state], [is, populous state], [is, the...","[[type, state], [most, populous_state], [type,..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...,"[[is, a state], [is, the second-most populous ...","[[type, state], [type, second-most_populous_st..."
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...,"[[is, French pronunciation], [compete, the top...","[[type, French_pronunciation], [compete, top_d..."


In [26]:
data.to_csv('ClCy-186.csv')

In [ ]:
pip install rfc3987

In [27]:
import rdflib
from rdflib import Graph
from rdflib.namespace import RDF
from rdflib import Literal, XSD
from rfc3987 import parse
flag = True
g = Graph()
k = 0
n = 0
for s,label,abstract,_,triples in data.values:
    
    if type(abstract) == float:
        continue
        
    for triple in triples:
        flag = True
        
        try:
            int(triple[1])
        except:
            flag = False 
    
        if flag is True:
            rel = triple[0].replace("'", '')
            rel = rel.replace('"', '')
            rel = rel.replace('`', '')
            rel = rel.replace('.', '')
            schema = ('http://schema.org/%s' %(rel))
            year = int(triple[1])
            year = Literal(year, datatype=XSD.gYear)
            try:
                parse(schema, rule='IRI')
            except:
                n+=1
                print(n,'bad -',triple)
                continue
            g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), year ))
            k+=1
        else:
            obj = triple[1].replace("'", '')
            obj = obj.replace('"', '')
            obj = obj.replace('`', '')
            obj = obj.replace('.', '')
            link = 'http://yago-knowledge.org/resource/%s' %(obj)
            if triple[0] == 'type':
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(link)
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), RDF.type, rdflib.URIRef(link)) )
                k+=1
            else:
                rel = triple[0].replace("'", '')
                rel = rel.replace('"', '')
                rel = rel.replace('`', '')
                rel = rel.replace('.', '')
                schema = ('http://schema.org/%s' %(rel))
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                try:
                    parse(schema, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), rdflib.URIRef(link)) )
                k+=1

1 bad - ['##nsWere', 'sovereign_nation']
2 bad - ['accountsOf', 'only_1.5%']
3 bad - ['isPronounced', 'pronounced_[prɪˈjəŋka_ˈtʃoːpɽa']
4 bad - ['##sMeet', 'MTV_reality_series']
5 bad - ['accountsOf', 'just_over_4.2%']


In [28]:
total = 0
for triples in data['ClCy']:
    if len(triples) == 0:
        continue 
    else:
        for triple in triples:
            total += 1
total,k

(3402, 3397)

In [29]:
n,total, n/total

(5, 3402, 0.0014697236919459142)

big - 314/111274 triples were not in proper format and needed to be deleted, thats 0.3%


small - 5/3402 trips removed, thats 0.15%

In [30]:
g.serialize(destination='clcy-186.ttl', format='turtle')

<Graph identifier=Nfb6e1baf99b047718e18a1944ba4a16d (<class 'rdflib.graph.Graph'>)>

In [31]:
max_trips = 0
for s,l,a,r,t in data.values:
    max_trips = max(len(t), max_trips)
    if max_trips == len(t):
        print(l,'-',len(t))

David Bowie - 28
Eminem - 35
France - 40
Winston Churchill - 44
George Washington - 46
